In [ ]:
import pandas as pd

In [ ]:
results = pd.read_csv("/Users/odeine/PycharmProjects/ilios-DocAI/src/outputs/output-new-metrics-gemini-baseline.csv")
results

In [ ]:
results.columns

In [ ]:
metrics_cols = [
    # 'levenshtein_distance',
    'bleu_score',
    'levenshtein_distance_scaled',
    # 'cosine_similarity',
    'meteor_score',
    'rouge1_precision_score',
    'rouge1_recall_score',
    # 'rougeL_f1_score',
    # 'rougeL_precision_score',
    # 'rougeL_recall_score'

]

In [ ]:
results_by_files = results[~results["file_name"].isna() & results["key_item"].isna()][["file_name"] + metrics_cols]
results_by_files

In [ ]:
results_by_key_items = results[~results["key_item"].isna() & results["file_name"].isna()][["key_item"] + metrics_cols]
results_by_key_items

In [ ]:
results_by_sample = results[~results["key_item"].isna() & ~results["file_name"].isna()]
results_by_sample

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for metric in metrics_cols:
    # Plot histogram
    sns.histplot(results_by_sample[metric], kde=True, bins=50)
    plt.title(f"Histogram of {metric} for files")
    plt.show()

In [ ]:
results_by_sample_sorted = results_by_sample[metrics_cols].sort_values("bleu_score", ascending=False)
import numpy as np
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
results_by_sample_sorted[metrics_cols] = scaler.fit_transform(results_by_sample_sorted[metrics_cols])

plt.figure(figsize=(20, 10))
for metric in metrics_cols:
    # Plot histogram
    plt.plot(np.arange(len(results_by_sample_sorted)), results_by_sample_sorted[metric])

plt.title(f"Sorted metrics for files")
plt.legend(metrics_cols)
plt.show()

In [ ]:
results_by_sample_sorted = results_by_sample[metrics_cols].sort_values("rouge1_precision_score", ascending=False)
import numpy as np
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
results_by_sample_sorted[metrics_cols] = scaler.fit_transform(results_by_sample_sorted[metrics_cols])

plt.figure(figsize=(20, 10))
for metric in metrics_cols:
    # Plot histogram
    plt.plot(np.arange(len(results_by_sample_sorted)), results_by_sample_sorted[metric])

plt.title(f"Sorted metrics for files")
plt.legend(metrics_cols)
plt.show()

In [ ]:
results_by_sample.sort_values("bleu_score", ascending=False).head(10)

In [ ]:
results_by_sample.sort_values("bleu_score", ascending=False).tail(10)

## Metrics calculation

In [ ]:
results_basic = pd.read_csv("/Users/odeine/PycharmProjects/ilios-DocAI/src/outputs/output-basic.csv")
results_few_shots = pd.read_csv("/Users/odeine/PycharmProjects/ilios-DocAI/src/outputs/output-few-shots.csv")

In [ ]:
import nltk

nltk.download("wordnet")

In [ ]:
from src.pipeline.utils import save_results
from src.validation.validation import calculate_metrics
import pandas as pd

results_basic = pd.read_csv("/Users/odeine/PycharmProjects/ilios-DocAI/src/outputs/output-basic.csv")
# results_basic = pd.read_csv("/Users/odeine/PycharmProjects/ilios-DocAI/src/outputs/output-few-shots.csv")

results_basic = results_basic.dropna(subset=["file_name", "key_item"]).rename({"key_item": "Key Items"}, axis=1)
# results_basic = results_basic.rename({"cosine_similarity": "cosine_similarity_777"}, axis=1)

results = []
for file_name in results_basic["file_name"].unique():
    predicted_actual = results_basic[results_basic["file_name"] == file_name][
        ["file_name", "Key Items", "Predicted Legal Terms", "Legal Terms"]]
    metrics = calculate_metrics(predicted_actual)
    predicted_actual_metrics = predicted_actual.merge(
            metrics, on="Key Items", how="outer"
        )
    
    results.append(predicted_actual_metrics)



save_results(results, "output-few-shots-new-metrics.csv")

In [ ]:
# from src.validation.validation import calculate_metrics
# 
# 
# 
# results_df = calculate_metrics(results_basic)
# results_df["file_name"] = results_basic["file_name"]
# 
# results_df = results_df.rename({"Key Items": "key_item"}, axis=1)
# metrics_total: pd.DataFrame = pd.DataFrame(results_df[metrics_cols].mean()).T
# metrics_total["file_name"] = np.nan
# metrics_total["key_item"] = np.nan
# 
# metrics_grouped_by_key_items: pd.DataFrame = (
#     results_df.groupby("key_item")[metrics_cols].mean().reset_index()
# )
# metrics_grouped_by_key_items["file_name"] = np.nan
# 
# metrics_grouped_by_file_name: pd.DataFrame = (
#     results_df.groupby("file_name")[metrics_cols].mean().reset_index()
# )
# metrics_grouped_by_file_name["key_item"] = np.nan
# 
# results_df = pd.concat(
#     [
#         pd.DataFrame(columns=results_df.columns),
#         metrics_total,
#         metrics_grouped_by_key_items,
#         metrics_grouped_by_file_name,
#         results_df,
#     ],
#     ignore_index=True,
# )

In [ ]:
# results_df